In [21]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3

# EDA

In [22]:
pd.set_option('display.max_columns', None)

In [31]:
conn = sqlite3.connect('FPA_FOD_20170508.sqlite')
raw_df = pd.read_sql_query("SELECT * from Fires", conn)

In [24]:
columns_to_drop = ['OBJECTID', 'FOD_ID', 'FPA_ID', 'NWCG_REPORTING_UNIT_ID',
                   'NWCG_REPORTING_UNIT_NAME', 'SOURCE_REPORTING_UNIT', 'SOURCE_REPORTING_UNIT_NAME', 'LOCAL_FIRE_REPORT_ID',
                   'LOCAL_INCIDENT_ID', 'FIRE_CODE', 'FIRE_NAME', 'ICS_209_INCIDENT_NUMBER', 'ICS_209_NAME', 'MTBS_ID', 'MTBS_FIRE_NAME','COMPLEX_NAME', 'OWNER_CODE', 'COUNTY', 'FIPS_CODE', 'FIPS_NAME', "Shape"]

# high correlation: OWNER_CODE
# feature engineering: NWCG_REPORTING_UNIT_ID, FOD_ID


* missing values in DISCOVERY_TIME, check correlation to fill missing values
* check if CONT_DATE is missing so same as DISCOVERY_DATE or if is correlated with fire size
* feature: length of fire. discovery minus contain. wh
* choose if date or DOY
* State- maybe do binning. a few big states, and 'other'
* fires occur specific DOY and specific time
* DOY vs LONGITUDE
* correlation stat_code and system_type
* feature ?


In [26]:
"""
* next:
* remove columns
* deal with missing values
* new features
* which features we should put our energy in
*
"""

'\n* next:\n* remove columns\n* deal with missing values\n* new features\n* which features we should put our energy in\n*\n'

Drop less important features

In [32]:
df = raw_df.drop(columns = columns_to_drop)

# Missing Values

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 626822 entries, 0 to 626821
Data columns (total 18 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   SOURCE_SYSTEM_TYPE     626822 non-null  object 
 1   SOURCE_SYSTEM          626822 non-null  object 
 2   NWCG_REPORTING_AGENCY  626822 non-null  object 
 3   FIRE_YEAR              626822 non-null  int64  
 4   DISCOVERY_DATE         626822 non-null  float64
 5   DISCOVERY_DOY          626822 non-null  int64  
 6   DISCOVERY_TIME         332746 non-null  object 
 7   STAT_CAUSE_CODE        626822 non-null  float64
 8   STAT_CAUSE_DESCR       626822 non-null  object 
 9   CONT_DATE              329734 non-null  float64
 10  CONT_DOY               329734 non-null  float64
 11  CONT_TIME              303098 non-null  object 
 12  FIRE_SIZE              626822 non-null  float64
 13  FIRE_SIZE_CLASS        626822 non-null  object 
 14  LATITUDE               626822 non-nu

## Features with missing values:
- DISCOVERY_TIME
- CONT_DATE
- CONT_DOY
- CONT_TIME

Their missing values are correlated with each other, along with "LOCAL_FRIE_REPORT_ID", "FIRE_CODE", "FIRE_NAME".
Let's take a look at the null values of them

In [43]:
discovery_time_missing = raw_df[raw_df["DISCOVERY_TIME"].isnull()].filter(items = ["DISCOVERY_TIME", "CONT_DATE", "CONT_DOY", "CONT_TIME", "LOCAL_FRIE_REPORT_ID" , "FIRE_CODE", "FIRE_NAME"])
discovery_time_missing

,DISCOVERY_TIME,CONT_DATE,CONT_DOY,CONT_TIME,FIRE_CODE,FIRE_NAME
0,None,NaN,NaN,None,None,None
3,None,NaN,NaN,None,None,None
4,None,NaN,NaN,None,None,None
8,None,NaN,NaN,None,None,None
10,None,NaN,NaN,None,None,None
...,...,...,...,...,...,...
626808,None,NaN,NaN,None,None,HWY 152 E/ RD 8
626809,None,NaN,NaN,None,None,None
626811,None,NaN,NaN,None,None,None
626820,None,NaN,NaN,None,None,BRANCIFORTE DR SOQUEL 5


In [51]:
cont_date_doy_missing = raw_df[raw_df["CONT_DATE"].isnull()].filter(items = ["LOCAL_FRIE_REPORT_ID", "CONT_DATE", "CONT_DOY", "CONT_TIME", "FIRE_CODE", "FIRE_NAME"])
cont_date_doy_missing


,CONT_DATE,CONT_DOY,CONT_TIME,FIRE_CODE,FIRE_NAME
0,NaN,NaN,None,None,None
3,NaN,NaN,None,None,None
4,NaN,NaN,None,None,None
6,NaN,NaN,None,None,None
7,NaN,NaN,None,None,None
...,...,...,...,...,...
626809,NaN,NaN,None,None,None
626811,NaN,NaN,None,None,None
626814,NaN,NaN,None,None,MS0 0808-3310326003
626820,NaN,NaN,None,None,BRANCIFORTE DR SOQUEL 5


In [52]:
cont_time_missing = raw_df[raw_df["CONT_TIME"].isnull()].filter(items = ["CONT_TIME", "LOCAL_FRIE_REPORT_ID", "CONT_DATE", "CONT_DOY", "FIRE_CODE", "FIRE_NAME"])
cont_time_missing

,CONT_TIME,CONT_DATE,CONT_DOY,FIRE_CODE,FIRE_NAME
0,None,NaN,NaN,None,None
3,None,NaN,NaN,None,None
4,None,NaN,NaN,None,None
6,None,NaN,NaN,None,None
7,None,NaN,NaN,None,None
...,...,...,...,...,...
626809,None,NaN,NaN,None,None
626811,None,NaN,NaN,None,None
626814,None,NaN,NaN,None,MS0 0808-3310326003
626820,None,NaN,NaN,None,BRANCIFORTE DR SOQUEL 5


It looks like we cant efficiently fill the missing values of one feature with the others.

Also, because they are so many (46% - 51%), imputing by mean

What about Linear Regression for each one, predicting their values?
good thing to check, but before, let us pre-process the values to fit in a model

